In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch as th
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corona-data2/test_dataset.pt
/kaggle/input/corona-data2/train_dataset.pt
/kaggle/input/data-dop2/Train_mental.pt
/kaggle/input/data-dop2/Test_mental.pt
/kaggle/input/pncr-metrics/poincare_metrics_runs (2).pkl
/kaggle/input/dop-data/Train_amazon.pt
/kaggle/input/dop-data/Train_mental.pt
/kaggle/input/dop-data/Test_amazon.pt
/kaggle/input/dop-data/Test_sentiment.pt
/kaggle/input/dop-data/Train_twtter.pt
/kaggle/input/dop-data/Test_twtter.pt
/kaggle/input/dop-data/Test_mental.pt
/kaggle/input/dop-data/Test_financial.pt
/kaggle/input/dop-data/Train_financial.pt
/kaggle/input/dop-data/Train_sentiment.pt
/kaggle/input/datasets-dop3/test_dataset.pt
/kaggle/input/datasets-dop3/Train_amazon.pt
/kaggle/input/datasets-dop3/Train_mental.pt
/kaggle/input/datasets-dop3/Test_amazon.pt
/kaggle/input/datasets-dop3/Test_sentiment.pt
/kaggle/input/datasets-dop3/Train_twtter.pt
/kaggle/input/datasets-dop3/Test_twtter.pt
/kaggle/input/datasets-dop3/Test_mental.pt
/kaggle/input/datasets-dop3/T

In [2]:
!pip install git+https://github.com/geoopt/geoopt.git

  Cloning https://github.com/geoopt/geoopt.git to /tmp/pip-req-build-lhu12yne
  Running command git clone --filter=blob:none --quiet https://github.com/geoopt/geoopt.git /tmp/pip-req-build-lhu12yne
  Resolved https://github.com/geoopt/geoopt.git to commit eaadc68fcae361778edf078b503ed79e4497c071
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━

In [3]:
import os
import sys
import pickle
import math
import time
from argparse import ArgumentParser

import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.utils as tutils
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from transformers import BertConfig
import geoopt as gt

import numpy as np
import pandas as pd
from tqdm import tqdm

2025-05-11 13:30:20.513288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746970220.692920      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746970220.744847      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import matplotlib.pyplot as plt

In [5]:
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score

In [6]:
class PoincareProbe(nn.Module):
    def __init__(
        self, device, default_dtype=th.float64, layer_num: int = 10, c: int = -1, 
        probe_dim = 32
    ):
        super().__init__()
        self.device = device
        self.default_dtype = default_dtype
        self.ball = gt.Stereographic(c)
        self.probe_dim = probe_dim
        self.layer_num = layer_num

        self.bound = 1 / math.sqrt(self.probe_dim)
        pos = th.zeros(self.probe_dim).uniform_(-self.bound, self.bound)
        neg = th.zeros(self.probe_dim).uniform_(-self.bound, self.bound)
        pos = self.ball.expmap0(pos)
        neg = self.ball.expmap0(neg)
        self.pos = gt.ManifoldParameter(data=pos, manifold=self.ball)
        self.neg = gt.ManifoldParameter(data=neg, manifold=self.ball)

        self.proj = nn.Parameter(data=th.zeros(768, self.probe_dim))
        self.trans = nn.Parameter(data=th.zeros(self.probe_dim, self.probe_dim))
        nn.init.uniform_(self.proj, -0.05, 0.05)
        nn.init.uniform_(self.trans, -0.05, 0.05)

    def forward(self, sequence_output):
        transformed = th.matmul(sequence_output, self.proj)
        transformed = self.ball.expmap0(transformed)
        transformed = self.ball.mobius_matvec(self.trans, transformed)
        pos_logits = self.ball.dist(self.neg, transformed).sum(-1)
        neg_logits = self.ball.dist(self.pos, transformed).sum(-1)

        return th.stack((neg_logits, pos_logits), dim=-1)

In [7]:
class EuclideanProbe(nn.Module):
    def __init__(
        self, device, default_dtype=th.float32, layer_num: int = 10,
        probe_dim = 32
    ):
        super().__init__()
        self.device = device
        self.default_dtype = default_dtype
        self.layer_num = layer_num

        self.probe_dim = probe_dim 
        self.bound = 1 / math.sqrt(self.probe_dim)
        self.pos = nn.Parameter(data=th.zeros(self.probe_dim))
        self.neg = nn.Parameter(data=th.zeros(self.probe_dim))
        nn.init.uniform_(self.pos, -self.bound, self.bound)
        nn.init.uniform_(self.neg, -self.bound, self.bound)
        self.tanh = nn.Tanh()
        self.proj = nn.Parameter(data=th.zeros(768, self.probe_dim))
        self.trans = nn.Parameter(data=th.zeros(self.probe_dim, self.probe_dim))
        nn.init.uniform_(self.proj, -0.05, 0.05)

    def forward(self, sequence_output):
        transformed = th.matmul(sequence_output, self.proj)
        transformed = self.tanh(transformed)
        # transformed = th.matmul(transformed, self.trans)
        pos_logits = (((self.neg - transformed) ** 2).sum(-1)).sum(-1)
        neg_logits = (((self.pos - transformed) ** 2).sum(-1)).sum(-1)
        return th.stack((neg_logits, pos_logits), dim=-1)

In [8]:
def Train(
    train_data_loader,
    probe,
    bert,
    loss_fct,
    optimizer,
    dev_data_loader=None,
    scheduler=None,
):
    # Train the probe
    probe.train()
    train_loss, dev_loss = 0, 0
    train_acc, dev_acc = 0, 0
    for batch in train_data_loader:
        optimizer.zero_grad()
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            batch[0],
            batch[1],
            batch[2],
            batch[3],
        )
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            text_input_ids.to(probe.device),
            text_token_type_ids.to(probe.device),
            text_attention_mask.to(probe.device),
            label.to(probe.device),
        )

        with th.no_grad():
            outputs = bert(
                text_input_ids,
                attention_mask=text_attention_mask,
                token_type_ids=text_token_type_ids,
                output_hidden_states=True,
            )
            hidden_states = outputs[2]
            sequence_output = (
                hidden_states[probe.layer_num].to(probe.device).to(probe.default_dtype)
            )

        logits = probe(sequence_output)
        l = loss_fct(logits.view(-1, 2), label.view(-1))
        train_loss += l.item()
        l.backward()
        optimizer.step()
        train_acc += (logits.argmax(-1) == label).sum().item()
    train_loss = train_loss / len(train_data_loader.dataset)
    train_acc = train_acc / len(train_data_loader.dataset)

    if dev_data_loader is not None:
        probe.eval()
        for batch in dev_data_loader:
            text_input_ids, text_token_type_ids, text_attention_mask, label = (
                batch[0],
                batch[1],
                batch[2],
                batch[3],
            )
            text_input_ids, text_token_type_ids, text_attention_mask, label = (
                text_input_ids.to(probe.device),
                text_token_type_ids.to(probe.device),
                text_attention_mask.to(probe.device),
                label.to(probe.device),
            )

            with th.no_grad():
                outputs = bert(
                    text_input_ids,
                    attention_mask=text_attention_mask,
                    token_type_ids=text_token_type_ids,
                    output_hidden_states=True,
                )
                hidden_states = outputs[2]
                sequence_output = (
                    hidden_states[probe.layer_num]
                    .to(probe.device)
                    .to(probe.default_dtype)
                )
                logits = probe(sequence_output)

                l = loss_fct(logits.view(-1, 2), label.view(-1))
                dev_loss += l.item()

            dev_acc += (logits.argmax(-1) == label).sum().item()
        # Adjust the learning rate
        if scheduler is not None:
            scheduler.step(dev_loss)

        dev_loss = dev_loss / len(dev_data_loader.dataset)
        dev_acc = dev_acc / len(dev_data_loader.dataset)

    return (
        train_loss,
        train_acc,
        dev_loss,
        dev_acc,
    )

In [11]:
def evaluate(test_data_loader, probe, bert, loss_fct):
    probe.eval()
    loss = 0
    acc = 0
    y_pred = [] 
    y_true = []
    for batch in test_data_loader:
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            batch[0],
            batch[1],
            batch[2],
            batch[3],
        )
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            text_input_ids.to(probe.device),
            text_token_type_ids.to(probe.device),
            text_attention_mask.to(probe.device),
            label.to(probe.device),
        )

        with th.no_grad():
            outputs = bert(
                text_input_ids,
                attention_mask=text_attention_mask,
                token_type_ids=text_token_type_ids,
                output_hidden_states=True,
            )
            hidden_states = outputs[2]
            sequence_output = (
                hidden_states[probe.layer_num].to(probe.device).to(probe.default_dtype)
            )
            logits = probe(sequence_output)

        l = loss_fct(logits.view(-1, 2), label.view(-1))
        loss += l.item()
        acc += (logits.argmax(-1) == label).sum().item()
        y_pred.extend(list(logits.argmax(-1).cpu().numpy()))
        y_true.extend(list(label.cpu().numpy()))
        
    return l / len(test_data_loader.dataset), acc / len(test_data_loader.dataset), np.array(y_true), np.array(y_pred)

In [12]:
default_dtype = th.float64
th.set_default_dtype(default_dtype)

bert_pretrained_file = "bert-base-uncased"

_layer_num = 10
_probe_dim = 32
_run_num = 1
_epoch_num = 10
_batch_size = 32
_stop_lr = 5e-8

In [13]:
def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    losses = []
    
    for batch in data_loader:
        # input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        # labels = batch['label'].to(device)
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            batch[0],
            batch[1],
            batch[2],
            batch[3],
        )
        text_input_ids, text_token_type_ids, text_attention_mask, label = (
            text_input_ids.to(device),
            text_token_type_ids.to(device),
            text_attention_mask.to(device),
            label.to(device),
        )
        outputs = model(
            input_ids=text_input_ids,
            attention_mask=text_attention_mask,
            labels=label
        )
        
        loss = outputs.loss
        losses.append(loss.item())
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    return np.mean(losses)

def fine_tune(bert, train_loader, device):
    EPOCHS = 3
    optimizer = optim.Adam(probe.parameters(), lr=0.001)
    total_steps = len(train_loader) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 10)
        
        train_loss = train_epoch(bert, train_loader, optimizer, device, scheduler)
        print(f'Train loss: {train_loss}')
        

In [14]:
def train_evaluate_Poincare(dataset, probe_dim=_probe_dim, layer_num=_layer_num, c=-1):
    if __name__ == "__main__":
    
        # device = th.device("cpu")
    
        if th.cuda.is_available():
            device = th.device("cuda")
            print("GPU доступен:", th.cuda.get_device_name(0))
        else:
            device = th.device("cpu")
            print("GPU недоступен, используется CPU")
    
        log_path = 'log'
        log_path_models = 'log/model'
    
        timestr = time.strftime("%m%d-%H%M%S")
        if not os.path.exists(log_path):
            os.makedirs(log_path)
    
        train_dataset = th.load(dataset['train'], weights_only=False)
        # dev_dataset = th.load(os.path.join(data_path, "dev_dataset.pt"))
        test_dataset = th.load(dataset['test'], weights_only=False)
    
        train_data_loader = DataLoader(train_dataset, batch_size=_batch_size, shuffle=True)
        # dev_data_loader = DataLoader(dev_dataset, batch_size=_batch_size, shuffle=False)
        test_data_loader = DataLoader(test_dataset, batch_size=_batch_size, shuffle=False)
    
        bert = BertModel.from_pretrained(bert_pretrained_file)
        # we are not fine-tuning BERT
        for param in bert.parameters():
            param.requires_grad = False
        bert.to(device)
    
        log_file = os.path.join(
            log_path, dataset['name'] + "_PUANCARE" + "-" + timestr + ".log"
        )
        avg_acc = []
        poincare_metric = defaultdict(list)
        
        for run in range(_run_num):
            probe = PoincareProbe(
                device=device, default_dtype=default_dtype, layer_num=layer_num,
                probe_dim=probe_dim, c=c
            )
            probe.to(device)
    
            loss_fct = nn.CrossEntropyLoss()
            # optimizer = optim.Adam(probe.parameters(), lr=0.001)
            optimizer = gt.optim.RiemannianAdam(
                        [
                            {"params": probe.proj},
                            {"params": probe.trans},
                            {"params": probe.pos},
                            {"params": probe.neg},
                        ],
                        lr=1e-3,
                    )
            scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=0)
    
            with open(log_file, "a") as f:
                f.write(f"Run: {run + 1}\n")
            for epoch in tqdm(range(_epoch_num), desc="[Epoch]"):
    
                start_time = time.time()
                train_loss, train_acc, dev_loss, dev_acc = Train(
                    train_data_loader,
                    probe,
                    bert,
                    loss_fct,
                    optimizer,
                    dev_data_loader=test_data_loader,
                    scheduler=scheduler,
                )
    
                secs = int(time.time() - start_time)
                mins = secs / 60
                secs = secs % 60
    
                test_loss, test_acc, y_true, y_pred = evaluate(test_data_loader, probe, bert, loss_fct)
    
                with open(log_file, "a") as f:
                    f.write(
                        f"Epoch: {epoch + 1} | time in {mins:.0f} minutes, {secs:.0f} seconds\n"
                    )
                    f.write(
                        f"\tTrain Loss: {train_loss:.4f}\t|\tTrain Acc: {train_acc * 100:.2f}%\n"
                    )
                    f.write(
                        f"\tDev Loss: {dev_loss:.4f}\t|\tDev Acc: {dev_acc * 100:.2f}%\n"
                    )
                    f.write(
                        f"\tTest Loss:  {test_loss:.4f}\t|\tTest Acc:  {test_acc * 100:.2f}%\n"
                    )
                    f.write("-" * 50 + "\n")

                poincare_metric['accuracy'].append(test_acc)
                poincare_metric['f1'].append(f1_score(y_true, y_pred))
                poincare_metric['precision'].append(precision_score(y_true, y_pred))
                poincare_metric['recall'].append(recall_score(y_true, y_pred))
                
                with open(log_file, "a") as f:
                    f.write(f"Avg Acc epoch {epoch}: {test_acc*100:.2f}%\n")
    
                if optimizer.param_groups[0]["lr"] < _stop_lr or epoch == _epoch_num - 1:
                        break
            if not os.path.exists(log_path_models):
                os.makedirs(log_path_models)
            probe_ckeckpoint = os.path.join(
                log_path_models,
                dataset['name'] + "_PUANCARE-" + "-" + timestr + ".pt",
            )
            th.save(probe.state_dict(), probe_ckeckpoint)
    
    return poincare_metric
        

In [21]:
datasets = [
    {'name': 'Amazon', 
    'train': '/kaggle/input/datasets-dop3/Train_amazon.pt', 
    'test': '/kaggle/input/datasets-dop3/Test_amazon.pt'},
    
    {'name': 'Financial', 
    'train': '/kaggle/input/datasets-dop3/Train_financial.pt', 
    'test': '/kaggle/input/datasets-dop3/Test_financial.pt'}, 
    
    {'name': 'Mental', 
    'train': '/kaggle/input/datasets-dop3/Train_mental.pt', 
    'test': '/kaggle/input/datasets-dop3/Test_mental.pt'}, 
    
    {'name': 'Twitter', 
    'train': '/kaggle/input/datasets-dop3/Train_twtter.pt', 
    'test': '/kaggle/input/datasets-dop3/Test_twtter.pt'},
    
    {'name': 'Sentiment', 
    'train': '/kaggle/input/datasets-dop3/Train_sentiment.pt', 
    'test': '/kaggle/input/datasets-dop3/Test_sentiment.pt'}, 

    {'name': 'Corobka Windows', 
    'train': '/kaggle/input/datasets-dop3/train_dataset.pt', 
    'test': '/kaggle/input/datasets-dop3/test_dataset.pt'},
    
]

In [16]:
with open('/kaggle/input/pncr-metrics/poincare_metrics_runs (2).pkl', 'rb') as f:
    poincare_metrics = pickle.load(f)

In [17]:
poincare_metrics

defaultdict(dict,
            {'Amazon': {'run 1': defaultdict(list,
                          {'accuracy': [0.866,
                            0.8596666666666667,
                            0.892,
                            0.8893333333333333,
                            0.8876666666666667,
                            0.8986666666666666,
                            0.899,
                            0.898,
                            0.8973333333333333,
                            0.898],
                           'f1': [0.8607068607068608,
                            0.8719196836020688,
                            0.8875000000000001,
                            0.8936579115951313,
                            0.8927093282394142,
                            0.9,
                            0.8988988988988987,
                            0.8992094861660079,
                            0.899214659685864,
                            0.8988095238095238],
                           'prec

In [17]:
# poincare_metrics = defaultdict(dict)
for dataset in datasets:
    print(dataset['name'] + '\n')
    for run in range(1, 6):
        print(f'run {run}' + '\n')
        poincare_metrics[dataset['name']][f'run {run}'] = train_evaluate_Poincare(dataset)

Twitter

run 1

GPU доступен: Tesla P100-PCIE-16GB


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

[Epoch]:  70%|███████   | 7/10 [20:07<08:37, 172.44s/it]


...

[Epoch]:  80%|████████  | 8/10 [10:16<02:34, 77.06s/it]


run 4

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  90%|█████████ | 9/10 [11:25<01:16, 76.20s/it]


run 5

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  90%|█████████ | 9/10 [11:25<01:16, 76.19s/it]


In [18]:
with open('poincare_metrics_runs.pkl', 'wb') as f:
    pickle.dump(poincare_metrics, f)

In [18]:
# Меняем размерность пробы 
poincare_metrics = defaultdict(dict)
for dataset in datasets:
    print(dataset['name'] + '\n')
    for probe_dim in [4, 8, 16, 64]:
        print(f'probe_dim {probe_dim}' + '\n')
        poincare_metrics[dataset['name']][f'probe_dim {probe_dim}'] = train_evaluate_Poincare(dataset, probe_dim)

Financial

probe_dim 4

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.60s/it]


probe_dim 8

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.56s/it]


probe_dim 16

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.51s/it]


probe_dim 64

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.52s/it]


In [31]:
# Меняем размерность пробы 
poincare_metrics = defaultdict(dict)
for dataset in datasets:
    print(dataset['name'] + '\n')
    for c in [-0.1, -0.5, -1, -2, -5]:
        print(f'c {c}' + '\n')
        poincare_metrics[dataset['name']][f'c {c}'] = train_evaluate_Poincare(dataset, c=c)

Financial

c -0.1

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:08<00:21, 21.42s/it]


c -0.5

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:08<00:21, 21.50s/it]


c -1

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:08<00:21, 21.48s/it]


c -2

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.52s/it]


c -5

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  86%|████████▌ | 6/7 [02:09<00:21, 21.56s/it]


In [21]:
with open('poincare_metrics_metrics_probes.pkl', 'wb') as f:
    pickle.dump(poincare_metrics, f)

In [20]:
def train_evaluate_Euclide(dataset, probe_dim=_probe_dim, layer_num=_layer_num):
    if __name__ == "__main__":
    
        # device = th.device("cpu")
    
        if th.cuda.is_available():
            device = th.device("cuda")
            print("GPU доступен:", th.cuda.get_device_name(0))
        else:
            device = th.device("cpu")
            print("GPU недоступен, используется CPU")
    
        log_path = 'log'
        log_path_models = 'log/model'
        
        timestr = time.strftime("%m%d-%H%M%S")
        if not os.path.exists(log_path):
            os.makedirs(log_path)
    
        train_dataset = th.load(dataset['train'], weights_only=False)
        # dev_dataset = th.load(os.path.join(data_path, "dev_dataset.pt"))
        test_dataset = th.load(dataset['test'], weights_only=False)
    
        train_data_loader = DataLoader(train_dataset, batch_size=_batch_size, shuffle=True)
        # dev_data_loader = DataLoader(dev_dataset, batch_size=_batch_size, shuffle=False)
        test_data_loader = DataLoader(test_dataset, batch_size=_batch_size, shuffle=False)
    
        bert = BertModel.from_pretrained(bert_pretrained_file)
        # we are not fine-tuning BERT
        for param in bert.parameters():
            param.requires_grad = False
        bert.to(device)
    
        log_file = os.path.join(
            log_path, dataset['name'] + "_Eucleude" + "-" + timestr + ".log"
        )
        avg_acc = []
        euclid_metric = defaultdict(list)
    
        for run in tqdm(range(_run_num), desc="[Run]"):
            probe = EuclideanProbe(
                device=device, default_dtype=default_dtype, layer_num=layer_num,
                probe_dim=probe_dim
            )
            probe.to(device)
    
            loss_fct = nn.CrossEntropyLoss()
            optimizer = optim.Adam(probe.parameters(), lr=0.001)
            
            scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=0)
    
            with open(log_file, "a") as f:
                f.write(f"Run: {run + 1}\n")
            for epoch in tqdm(range(_epoch_num), desc="[Epoch]"):
    
                start_time = time.time()
                train_loss, train_acc, dev_loss, dev_acc = Train(
                    train_data_loader,
                    probe,
                    bert,
                    loss_fct,
                    optimizer,
                    dev_data_loader=test_data_loader,
                    scheduler=scheduler,
                )
    
                secs = int(time.time() - start_time)
                mins = secs / 60
                secs = secs % 60
    
                test_loss, test_acc, y_true, y_pred = evaluate(test_data_loader, probe, bert, loss_fct)
    
                with open(log_file, "a") as f:
                    f.write(
                        f"Epoch: {epoch + 1} | time in {mins:.0f} minutes, {secs:.0f} seconds\n"
                    )
                    f.write(
                        f"\tTrain Loss: {train_loss:.4f}\t|\tTrain Acc: {train_acc * 100:.2f}%\n"
                    )
                    f.write(
                        f"\tDev Loss: {dev_loss:.4f}\t|\tDev Acc: {dev_acc * 100:.2f}%\n"
                    )
                    f.write(
                        f"\tTest Loss:  {test_loss:.4f}\t|\tTest Acc:  {test_acc * 100:.2f}%\n"
                    )
                    f.write("-" * 50 + "\n")
    
                euclid_metric['accuracy'].append(test_acc)
                euclid_metric['f1'].append(f1_score(y_true, y_pred))
                euclid_metric['precision'].append(precision_score(y_true, y_pred))
                euclid_metric['recall'].append(recall_score(y_true, y_pred))
                with open(log_file, "a") as f:
                    f.write(f"Avg Acc epoch {epoch}: {test_acc*100:.2f}%\n")
                    
                if optimizer.param_groups[0]["lr"] < _stop_lr or epoch == _epoch_num - 1:
                    break
            
            if not os.path.exists(log_path_models):
                os.makedirs(log_path_models)
            probe_ckeckpoint = os.path.join(
                log_path_models,
                dataset['name'] + "_Eucleude" + "-run-" + str(run) + "-" + timestr + ".pt",
            )
            th.save(probe.state_dict(), probe_ckeckpoint)
    
    return euclid_metric

In [ ]:
euclide_metrics = defaultdict(dict)
for dataset in datasets:
    for run in range(1, 6):
        print(f'run {run}' + '\n')
        euclide_metrics[dataset['name']][f'run {run}'] = train_evaluate_Euclide(dataset)

run 1

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [25:40<00:00, 1540.15s/it]


run 2

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [25:39<00:00, 1539.67s/it]


run 3

GPU доступен: Tesla P100-PCIE-16GB


...

[Run]: 100%|██████████| 1/1 [03:11<00:00, 191.64s/it]


run 3

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [03:11<00:00, 191.71s/it]


run 4

GPU доступен: Tesla P100-PCIE-16GB


[Epoch]:  80%|████████  | 8/10 [02:33<00:38, 19.17s/it]

In [ ]:
with open('euclide_metrics_runs.pkl', 'wb') as f:
    pickle.dump(euclide_metrics, f)

In [23]:
euclide_metrics = defaultdict(dict)
for dataset in datasets:
    for probe_dim in [4, 8, 16, 64]:
        print(f'probe_dim {probe_dim}' + '\n')
        euclide_metrics[dataset['name']][f'probe_dim {probe_dim}'] = train_evaluate_Euclide(dataset, probe_dim)

probe_dim 4

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:05<00:00, 125.20s/it]


probe_dim 8

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:05<00:00, 125.73s/it]


probe_dim 16

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:05<00:00, 125.67s/it]


probe_dim 64

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:05<00:00, 125.65s/it]


In [21]:
# Меняем размерность пробы 
euclide_metrics = defaultdict(dict)
for dataset in datasets:
    print(dataset['name'] + '\n')
    for layer_num in range(3, 12):
        print(f'layer_num {layer_num}' + '\n')
        euclide_metrics[dataset['name']][f'layer_num {layer_num}'] = train_evaluate_Euclide(dataset, layer_num=layer_num)

Financial

layer_num 3

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:04<00:00, 124.66s/it]


layer_num 4

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:04<00:00, 124.67s/it]


layer_num 5

GPU доступен: Tesla P100-PCIE-16GB


...

[Run]: 100%|██████████| 1/1 [02:04<00:00, 124.58s/it]


layer_num 10

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:04<00:00, 124.56s/it]


layer_num 11

GPU доступен: Tesla P100-PCIE-16GB


[Run]: 100%|██████████| 1/1 [02:04<00:00, 124.65s/it]
